In [38]:
yolo_dir = "yolo1.pt"
keras_dir = "model_23_may_2025.keras"

In [12]:
from ultralytics import YOLO
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import json
import os

In [ ]:

# from google.colab.patches import cv2_imshow

def enlarge_bbox(x1, y1, x2, y2, scale=1.2, img_width=640, img_height=640):
    # Hitung center dan ukuran
    w = x2 - x1
    h = y2 - y1
    cx = x1 + w / 2
    cy = y1 + h / 2

    # Perbesar
    new_w = w * scale
    new_h = h * scale

    # Hitung koordinat baru
    new_x1 = max(0, int(cx - new_w / 2))
    new_y1 = max(0, int(cy - new_h / 2))
    new_x2 = min(img_width, int(cx + new_w / 2))
    new_y2 = min(img_height, int(cy + new_h / 2))

    return new_x1, new_y1, new_x2, new_y2

# Load best model
# model_yolo = YOLO("best.pt")

In [32]:
def detect_and_crop(img_path, model_path):
    
    model_yolo = YOLO(model_path)
    results = model_yolo.predict(
        source=img_path,
        conf=0.4,
        save=True,  # Jangan langsung simpan dulu
        imgsz=640,
        device='cpu'
    )

    # Load original image
    img_path = img_path
    img = cv2.imread(img_path)
    img_height, img_width = img.shape[:2]

    base_dir = "hasil_crop"

    # Buat base_dir jika belum ada
    os.makedirs(base_dir, exist_ok=True)

    # Ambil daftar folder yang sudah ada di base_dir
    existing_folders = [name for name in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, name))]

    # Buat nama folder baru dengan angka urutan (misalnya: 0, 1, 2, ...)
    folder_index = len(existing_folders)
    folder_dir = os.path.join(base_dir, str(folder_index))

    # Buat folder baru
    os.makedirs(folder_dir, exist_ok=True)

    # Ambil hasil prediksi pertama
    boxes = results[0].boxes.xyxy.cpu().numpy()  # Format: [x1, y1, x2, y2]
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box[:4]
        x1, y1, x2, y2 = enlarge_bbox(x1, y1, x2, y2, scale=1.75, img_width=img_width, img_height=img_height)
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Crop objek dan simpan
        crop = img[y1:y2, x1:x2]
        scale_factor = 4.5
        resized_crop = cv2.resize(crop, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)
        crop_path = f'{folder_dir}/ultralytics_crop_{i}.jpg'
        cv2.imwrite(crop_path, resized_crop)
    return folder_dir 
# Tampilkan gambar
# cv2.imshow("Detection Results", img)
# cv2_imshow(img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [15]:
with open("class_indices.json", "r") as f:
    class_indices = json.load(f)
class_names = [None] * len(class_indices)

for names, index in class_indices.items():
  class_names[index] = names

len(class_names)

15

In [26]:
def predict_image(img_path, model_keras_path):
  model_keras = keras.models.load_model(model_keras_path)
  img = load_img(img_path, target_size=(128, 128))  # sesuaikan ukuran dengan model kamu
  img_array = img_to_array(img)
  img_array = img_array / 255.0  # normalisasi jika model dilatih dengan input ter-normalisasi
  img_array = np.expand_dims(img_array, axis=0)  # tambahkan batch dimension

  # 3. Prediksi
  # print(img_array.shape)
  pred = model_keras.predict(img_array)
  pred_index = np.argmax(pred)

  # # 4. Tampilkan hasil prediksi
  print("Output prediksi:", class_names[pred_index])

In [30]:
def inference(image, yolo, keras):
    count = 0
    deteksi_dir = detect_and_crop(image, yolo)
    for file_name in os.listdir(deteksi_dir):
        if file_name.endswith(".jpg"):
            file_path = os.path.join(deteksi_dir, file_name)
            print(f"Memproses {file_path}...")
            predict_image(file_path, keras)
            count += 1
    print(f"Total gambar yang diproses: {count}")
    

In [39]:
inference("Acne-C.jpg", yolo_dir, keras_dir)


image 1/1 c:\dicoding\capstone\test_final\test_final\Acne-C.jpg: 448x640 5 Acnes, 68.0ms
Speed: 2.2ms preprocess, 68.0ms inference, 3.4ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs\detect\predict10
Memproses hasil_crop\9\ultralytics_crop_0.jpg...
1/1 [==============================] - 1s 673ms/step
Output prediksi: Milium
Memproses hasil_crop\9\ultralytics_crop_1.jpg...
1/1 [==============================] - 1s 898ms/step
Output prediksi: Milium
Memproses hasil_crop\9\ultralytics_crop_2.jpg...
1/1 [==============================] - 1s 609ms/step
Output prediksi: Folliculitis
Memproses hasil_crop\9\ultralytics_crop_3.jpg...
1/1 [==============================] - 1s 603ms/step
Output prediksi: Milium
Memproses hasil_crop\9\ultralytics_crop_4.jpg...
1/1 [==============================] - 1s 640ms/step
Output prediksi: Acne
Total gambar yang diproses: 5
